# Orbital Demodulation: CTCV J2056-3014
### *To view plots, visit [nbviewer](https://nbviewer.org/github/ericymiao/ctcvj2056-timing-spin-evolution/blob/main/notebooks/06_demodulation.ipynb)*

Search for orbital period through timing demodulation.

## What is Orbital Demodulation?

If the X-ray source is in a binary, the photon arrival times are modulated by the orbital motion. As the source moves toward us, photons arrive slightly early; as it moves away, they arrive late. This causes the apparent spin period to vary over the orbit, reducing the coherence of the pulsation signal.

**Demodulation** corrects for this effect by shifting each photon's arrival time by the expected orbital delay:
$$\Delta t = A \sin\left(\frac{2\pi t}{T_{orb}} + \phi\right)$$

where:
- **A** = amplitude of timing delay = (a sin i)/c, the projected semi-major axis in light-seconds
- **T_orb** = orbital period
- **φ** = orbital phase at reference time

When the correct orbital parameters are applied, the pulsation signal becomes more coherent and the Z² power increases.

In [1]:
import sys
sys.path.insert(0, '../src')

import numpy as np
from stingray import EventList
from stingray.pulse import z_n_search
import hendrics.io as HENio

from bokeh.plotting import figure, show
from bokeh.layouts import column, row, gridplot
from bokeh.io import output_notebook
from bokeh.models import Title
from timing_analysis import *

from plotting import set_axis_styles
from demodulation import (
    delta_t, shifted_times, demodulate_grid_search, 
    find_best_parameters, project_to_period_axis, shifted_times_scramble
)

output_notebook()

Loading BokehJS ...

## Load Data

In [2]:
events = {}
data_path = '../../../analysis_files_strictfilter'

events['4592010200'] = HENio.load_events(f'{data_path}/ni4592010200_0mpu7_cl_bary_nicer_xti_ev_calib.nc')
events['7656010100'] = HENio.load_events(f'{data_path}/ni7656010100_0mpu7_cl_bary_nicer_xti_ev_calib.nc')

for obs in events:
    events[obs].filter_energy_range([0.3, 10], inplace=True)
    total_time = np.sum(events[obs].gti[:, 1] - events[obs].gti[:, 0])
    print(f"{obs} total time: {total_time/1000:.2f} ks, events: {len(events[obs].time):,}")

4592010200 total time: 47.61 ks, events: 95,241
7656010100 total time: 40.09 ks, events: 170,094


## Unmodulated Z^2 Power

First, we measure the Z² power without any orbital correction. This serves as the baseline - if demodulation with certain orbital parameters produces higher Z² power, that suggests those parameters are closer to the true orbit.

In [3]:
P_spin = 29.60968584  # Spin period in seconds
nharm = 1

unmodulated_Z = {}
for obs in events:
    _, z = z_n_search(events[obs].time, 1/P_spin, nharm=nharm)
    unmodulated_Z[obs] = float(z)
    #print(f"{obs} unmodulated Z^2_{nharm}: {z:.1f}")

/var/folders/vt/3pcpgl9s1m9dr65kms8vmpth0000gn/T/ipykernel_53481/3941947518.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  unmodulated_Z[obs] = float(z)


## Demodulation Grid Search

We search over a 3D grid of orbital parameters (T, A, φ):
- **T (orbital period)**: Range chosen based on prior constraints (e.g., from optical observations, the expected ~6320 s period)
- **A (timing amplitude)**: From 0 (no orbital motion) to ~1 second (typical for CVs)
- **φ (orbital phase)**: Full range 0 to 2π

For each grid point, we:
1. Shift all photon times by the model delay
2. Calculate Z² at the known spin frequency
3. Record the power

The grid point with maximum Z² gives the best-fit orbital parameters. This is computationally expensive (N_T × N_A × N_φ evaluations), but parallelizable.

In [4]:
# Define search grid
N_T = 150   # Number of orbital periods to search
N_A = 30    # Number of amplitudes
N_phi = 50  # Number of phases

Ts = np.linspace(3000, 6600, N_T)      # Orbital period range (seconds)
As = np.linspace(0, 1, N_A)            # Amplitude range (seconds)
phis = np.linspace(0, 2*np.pi, N_phi)  # Phase range (radians)

print(f"Grid: {N_T} x {N_A} x {N_phi} = {N_T*N_A*N_phi:,} evaluations per observation")

Grid: 150 x 30 x 50 = 225,000 evaluations per observation


In [31]:
# Run demodulation (WARNING: This can take a long time!)
# Uncomment to run, or load pre-computed results

"""results = {}
for obs in events:
    print(f"\nProcessing {obs}...")
    results[obs] = demodulate_grid_search(
        events[obs].time, P_spin, Ts, As, phis, nharm=nharm
    )
np.save('../data/demodulation_results_3000-6500.npy', results)"""

Loaded pre-computed demodulation results


In [5]:
# Load pre-computed results
results = np.load('../data/demodulation_results_6100-6500.npy', allow_pickle=True).item()
Ts = np.linspace(6100, 6600, N_T)

## Analyze Results

We project the 3D grid results onto the orbital period axis by taking the maximum Z² power over all (A, φ) at each T. The resulting curve shows:

- **Peak at true orbital period**: If orbital modulation is present, Z² should peak near the true T_orb
The expected orbital period from optical observations (~6320 s) is marked for comparison.

**Results (WIP)**: While there is a peak near the orbital period, we see it that the Z² increase it provides is well within the uncertainty range of the detected power (calculated by inverting the non-central chi² CDF, as the powers in a Z² periodogram are described by a non-central chi² distribution (Vaughan et al. 1994))
- (Is this correct analysis?)

In [15]:
if results is not None:
    plots = []
    
    for obs in results:
        # Project onto orbital period axis
        max_powers = project_to_period_axis(results[obs], Ts)
        
        p = figure(width=600, height=400,
                   x_axis_label='Orbital Period (s)',
                   y_axis_label='Max Z^2 Power',
                   title=Title(text=obs, text_font_size="14pt", align="center"))
        set_axis_styles(p)
        
        p.scatter(Ts, max_powers, size=4)
        p.line(Ts, max_powers)
        
        # Mark unmodulated power
        p.hspan(unmodulated_Z[obs], alpha=0.8, line_color='red', 
                legend_label='Unmodulated')
        print(z2_confidence_interval(unmodulated_Z[obs], 1, 2))

        p.hspan(z2_confidence_interval(unmodulated_Z[obs], 1, sigma_level=2)[1], line_color = "green")
        p.hspan(z2_confidence_interval(unmodulated_Z[obs], 1, sigma_level=1)[1], line_color = "green")
        p.hspan(z2_confidence_interval(unmodulated_Z[obs], 1, sigma_level=0.5)[1], line_color = "green")
        
        # Mark expected orbital period (~6320 s from optical)
        p.vspan(6320, alpha=0.3, line_color='blue', 
                legend_label='Expected Porb')
        
        # Find best parameters
        best = find_best_parameters(results[obs], Ts, As, phis)
        print(f"{obs}: Best T = {best['T']:.1f} s, A = {best['A']:.3f} s, Z^2 = {best['z2_max']:.1f}")
        
        plots.append(p)
    
    show(row(plots))


(np.float64(872.4173017650012), np.float64(1124.6433707054705))
4592010200: Best T = 6412.1 s, A = 0.655 s, Z^2 = 1006.7
(np.float64(1192.1619868702294), np.float64(1484.325712174771))
7656010100: Best T = 6251.0 s, A = 1.000 s, Z^2 = 1373.7


In [ ]:
# Demodulation test, but scramble the delta_t's. 
# For peak near orbital period in 7656010100, and "bump" peak in 4592010000. 

# Test parameters   
sim2_N_Trials = 1000
sim2_N_A = 40
sim2_N_phi = 50

sim2_As = np.linspace(0, 1, sim2_N_A)
sim2_phis = np.linspace(0, 2*np.pi, sim2_N_phi)

P_orb = 29.609686

# Initialize results dictionary
sim2_results = {}
total_permut = 0

# Create arrays for each observation
for obs in ['4592010200', '7656010100']:
    sim2_results[obs] = np.zeros((sim2_N_Trials, sim2_N_A, sim2_N_phi))
    event = events[obs]
    t = event.time

    if(obs == '4592010200'): P_mod = 6347
    else: P_mod = 6226.75

    for p in range(sim2_N_Trials):
        inda = 0
        for a in sim2_As:
            indphi = 0
            for phi in sim2_phis:
                t_shifted = shifted_times_scramble(phi, a, t, P_mod)
                f, z = z_n_search(t_shifted, 1/P_orb, nharm=2)
                sim2_results[obs][p, inda, indphi] = z
                indphi += 1
            inda += 1
        total_permut += 1
        print(str(100*(total_permut/sim2_N_Trials)) + "%")

np.save('sim2_results.npy', sim2_results)
print("Results saved to sim2_results.npy")

In [ ]:
sim2_results = np.load('../data/sim2_results.npy', allow_pickle=True).item()

# Observed peak Z^2 from real demodulation (from cell-11 results)
observed_peak = {
    '4592010200': 1006.7,
    '7656010100': 1373.7
}

obs = '7656010100'  # Choose which observation to analyze

# Compute max Z^2 for each scrambled trial
n_trials = sim2_results[obs].shape[0]
max_zs = np.array([np.max(sim2_results[obs][i, :, :]) for i in range(n_trials)])

# Plot 1: Max Z^2 vs trial number
p1 = figure(
    width=600, height=400,
    x_axis_label='Trial', y_axis_label='Max Z²',
    title=Title(text=f"Scrambled Demodulation: {obs}", text_font_size="14pt", align="center")
)
set_axis_styles(p1)
p1.scatter(np.arange(n_trials), max_zs, size=3, alpha=0.5)
p1.hspan(observed_peak[obs], line_color='red', line_width=2, legend_label=f'Observed: {observed_peak[obs]:.1f}')

# Plot 2: Histogram of scrambled max Z^2 values
p2 = figure(
    width=600, height=400,
    x_axis_label='Max Z²', y_axis_label='Count',
    title=Title(text=f"Null Distribution: {obs}", text_font_size="14pt", align="center")
)
set_axis_styles(p2)

hist, edges = np.histogram(max_zs, bins=40)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="skyblue", line_color="white")
p2.vspan(observed_peak[obs], line_color='red', line_width=2, legend_label=f'Observed: {observed_peak[obs]:.1f}')

# Compute significance
n_exceed = np.sum(max_zs >= observed_peak[obs])
p_value = n_exceed / n_trials
print(f"Observed peak: {observed_peak[obs]:.1f}")
print(f"Scrambled mean: {np.mean(max_zs):.1f} ± {np.std(max_zs):.1f}")
print(f"Trials exceeding observed: {n_exceed}/{n_trials} (p = {p_value:.3f})")

show(row(p1, p2))